In [3]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No API key was found')
elif not api_key.startswith("sk-proj-"):
    print("API key is found but is not in the proper format")
else:
    print("API key found and looks good so far")

API key found and looks good so far


In [5]:
openai = OpenAI()

In [16]:
system_prompt = """You are a research summarizer. That summarizes the content of the research paper in no more than 1000 words. The research summary that you provide should include the following:
1) Title and Authors - Identify the study and contributors.
2) Objective/Problem - State the research goal or question.
3) Background - Briefly explain the context and significance.
4) Methods - Summarize the approach or methodology.
5) Key Findings - Highlight the main results or insights.
6) Conclusion - Provide the implications or contributions of the study.
7) Future Directions - Suggest areas for further research or exploration.
8) Limitations - Highlight constraints or challenges in the study.
9) Potential Applications - Discuss how the findings can be applied in real-world scenarios.
Keep all points concise, clear, and focused and generate output in markdown."""

In [2]:
!pip install ipywidgets pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.5 MB/s eta 0:00:00a 0:00:01


In [8]:
import pdfplumber
from ipywidgets import widgets
from io import BytesIO

In [9]:
from IPython.display import display, Markdown

In [10]:
def messages_for(user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [11]:
def summarize(user_prompt):
    # Generate messages using the user_prompt
    messages = messages_for(user_prompt)
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Correct model name
            messages=messages,
            max_tokens = 1000 # Pass the generated messages
        )
        # Return the content from the API response correctly
        return response.choices[0].message.content
    except Exception as e:
        # Instead of printing, return an error message that can be displayed
        return f"Error in OpenAI API call: {e}"

In [12]:
upload_widget = widgets.FileUpload(
    accept='.pdf', 
    multiple=False,
    description='Upload PDF',
    layout=widgets.Layout(width='300px',height = '100px', border='2px dashed #cccccc', padding='10px')
)

In [17]:
def extract_text_and_generate_summary(change):
    print("extracting text")
    if upload_widget.value:
        # Extract the first uploaded file
        uploaded_file = list(upload_widget.value)[0]
        pdf_file = uploaded_file['content']

        # Extract text from the PDF
        try:
            with pdfplumber.open(BytesIO(pdf_file)) as pdf:
                extracted_text = "\n".join(page.extract_text() for page in pdf.pages)

            # Generate the user prompt
            user_prompt = (
                f"You are looking at the text from a research paper. Summarize it in no more than 1000 words. "
                f"The output should be in markdown.\n\n{extracted_text}"
            )

            # Get the summarized response
            response = summarize(user_prompt)
            
            if response:
                # Use IPython's display method to show markdown below the cell
                display(Markdown(response))
        
        except Exception as e:
            # If there's an error, display it using Markdown
            display(Markdown(f"**Error:** {str(e)}"))

    # Reset the upload widget
    upload_widget.value = ()

In [18]:
upload_widget.observe(extract_text_and_generate_summary, names='value')

In [19]:
display(upload_widget)

FileUpload(value=(), accept='.pdf', description='Upload PDF', layout=Layout(border_bottom='2px dashed #cccccc'…